In [86]:
import numpy as np
import random

In [85]:
FILE = "fashion-mnist_train.csv"
with open(f"data/{FILE}") as f:
    examples = f.read().strip().split("\n")[1:]

train_data, valid_data, test_data = [], [], []
valid_cutoff = .8
test_cutoff = .9

for example in examples:
    label, pixels = example.split(",", 1)
    label = int(label)
    if label > 4:
        continue
    pixels = [int(p) for p in pixels.split(",")]
    data = np.asarray(pixels, dtype="int32")
    data = data.reshape((1,28,28))
    # Scale values from -1 to 1
    data = ((data / 255) - .5) / .5
    # Move channel axis to first axis
    #data = np.moveaxis(data, -1, 0)
    target = np.zeros((5))
    target[label] = 1
    row = (data, target, )

    split = random.random()
    if split > test_cutoff:
        test_data.append(row)
    elif split > valid_cutoff:
        valid_data.append(row)
    else:
        train_data.append(row)

In [37]:
import torch

device = torch.device("cpu")

from torchvision.io import read_image
from torchvision import transforms as T
from torch.utils.data import Dataset
import math

class ImageDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.normalize = T.Compose([
            T.ConvertImageDtype(torch.float)
        ])

    def __len__(self):
        return len(self.dataset)

    def classes(self):
        return 5

    def __getitem__(self, idx):
        data, target = self.dataset[idx]
        data = torch.from_numpy(data)
        return self.normalize(data), target

In [38]:
from torch.utils.data import DataLoader
from torch import nn

BATCH_SIZE = 1
EPOCHS = 20

train_dataset = ImageDataset(train_data)
valid_dataset = ImageDataset(valid_data)
test_dataset = ImageDataset(test_data)

train = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)
test = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [83]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(1, 1, 3),
            nn.ReLU(True),
        )

        self.dense = nn.Sequential(
            nn.Linear(26 ** 2, 5)
        )

    def forward(self, x):
        x = self.cnn(x)
        x = torch.flatten(x, 1)
        x = self.dense(x)
        return x

In [84]:
model = NeuralNetwork().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=5e-3)

size = len(train.dataset)

for epoch in range(EPOCHS):
    for batch, (images, targets) in enumerate(train):
        optimizer.zero_grad()

        images = images.to(device)
        pred = model(images.float())

        loss = loss_fn(pred, targets)

        loss.backward()
        optimizer.step()

    loss = loss.item()
    print(f"Epoch {epoch} train loss: {loss}")

    match = list()

    with torch.no_grad():
        for batch, (images, targets) in enumerate(valid):
            images = images.to(device)
            outputs = model(images.float())

            _, p = torch.max(outputs.data, 1)

            p = p.cpu().numpy()
            match.append(p[0] == np.argmax(targets, axis=1)[0])

    print(f"Valid accuracy: {sum(match) / len(match)}")

Epoch 0 train loss: 0.13648992776870728
Valid accuracy: 0.8597640991210938
Epoch 1 train loss: 0.24475865066051483
Valid accuracy: 0.8725426197052002
Epoch 2 train loss: 0.13204717636108398
Valid accuracy: 0.8633682727813721
Epoch 3 train loss: 0.0027682576328516006
Valid accuracy: 0.8574705123901367
Epoch 4 train loss: 0.06656605750322342
Valid accuracy: 0.8797509670257568
Epoch 5 train loss: 0.711868941783905
Valid accuracy: 0.8673001527786255
Epoch 6 train loss: 1.0944929122924805
Valid accuracy: 0.8617300391197205
Epoch 7 train loss: -0.0
Valid accuracy: 0.8620576858520508
Epoch 8 train loss: 0.05632047355175018
Valid accuracy: 0.8650065660476685
Epoch 9 train loss: 0.0006551980040967464
Valid accuracy: 0.8705766797065735
Epoch 10 train loss: 0.021929049864411354
Valid accuracy: 0.8764744400978088
Epoch 11 train loss: 0.0053314645774662495
Valid accuracy: 0.8718872666358948
Epoch 12 train loss: 0.07228444516658783
Valid accuracy: 0.8751638531684875
Epoch 13 train loss: 0.0031007090